In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import h2o
from h2o.automl import H2OAutoML
h2o.init(nthreads=7)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,07 secs
H2O cluster timezone:,Europe/Paris
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,24 days
H2O cluster name:,H2O_from_python_edu_r8114u
H2O cluster total nodes:,1
H2O cluster free memory:,1.887 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,7
H2O cluster status:,"locked, healthy"


In [3]:
runtime = 300

In [4]:
train_high = h2o.import_file('../cleaned-data/08-train_high-colorAsInt-restAsDummies.csv')
train_low = h2o.import_file('../cleaned-data/08-train_low-colorAsInt-restAsDummies.csv')
test_high = h2o.import_file('../cleaned-data/08-test_high-colorAsInt-restAsDummies.csv')
test_low = h2o.import_file('../cleaned-data/08-test_low-colorAsInt-restAsDummies.csv')      
display(train_high)
test_high

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


id,carat,color,depth,table,x,y,z,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,price
8,2.05,7,62.4,57,8.17,8.09,5.08,0,0,1,0,0,0,0,1,0,0,0,0,0,18193
34,2.01,6,64.9,60,7.82,7.78,5.06,1,0,0,0,0,0,0,0,0,0,1,0,0,13744
42,2,5,64.2,52,7.91,7.88,5.07,0,1,0,0,0,0,0,0,1,0,0,0,0,11400
72,2,5,60.3,56,8.08,8.11,4.88,0,1,0,0,0,0,0,0,1,0,0,0,0,16425
82,2.28,3,62.7,57,8.43,8.36,5.26,0,0,1,0,0,0,0,0,0,0,1,0,0,16450
93,2.02,1,60.4,58,8.2,8.16,4.94,0,0,0,0,1,0,0,1,0,0,0,0,0,17357
141,2,1,60.7,60,8.07,8.11,4.91,0,0,0,1,0,0,0,0,1,0,0,0,0,15312
154,2.01,5,61.5,62,8.07,7.94,4.92,0,0,0,1,0,0,0,0,0,1,0,0,0,15475
165,2.31,6,62.9,58,8.43,8.34,5.27,0,0,0,1,0,0,0,0,1,0,0,0,0,17218
167,2.01,5,60.6,57,8.19,8.09,4.92,0,0,1,0,0,0,0,1,0,0,0,0,0,17383


id,carat,color,depth,table,x,y,z,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
2,2.03,4,61.9,59,8.14,8.09,5.02,0,0,0,1,0,0,0,1,0,0,0,0,0
29,2.52,4,63.7,59,8.57,8.52,5.44,0,1,0,0,0,0,0,0,1,0,0,0,0
38,2,5,58.8,61,8.29,8.23,4.86,0,0,0,1,0,0,0,1,0,0,0,0,0
75,2.01,4,60.3,59,8.23,8.15,4.94,0,0,0,1,0,0,0,1,0,0,0,0,0
76,2.15,2,62.5,57,8.2,8.16,5.11,0,0,1,0,0,0,0,1,0,0,0,0,0
90,2.02,2,61.7,57,8.12,8.05,4.99,0,0,1,0,0,0,0,0,1,0,0,0,0
116,2.02,4,61.8,58,8.16,8.1,4.94,0,0,0,1,0,0,0,1,0,0,0,0,0
143,2.11,5,63.5,55,8.08,8.13,5.15,0,0,0,0,1,0,0,0,1,0,0,0,0
189,2.05,1,62.5,60,8.09,8.14,5.07,0,0,0,1,0,0,0,0,0,1,0,0,0
242,2.54,3,63.5,56,8.68,8.65,5.5,0,0,0,0,1,0,0,0,1,0,0,0,0


In [5]:
list(map(lambda x: print(x.shape),[train_high, train_low, test_high, test_low]))

(1615, 22)
(38730, 22)
(521, 21)
(12928, 21)


[None, None, None, None]

In [6]:
xTrainCols = train_high.col_names[1:-1]
xTrainCols

['carat',
 'color',
 'depth',
 'table',
 'x',
 'y',
 'z',
 'cut_Fair',
 'cut_Good',
 'cut_Ideal',
 'cut_Premium',
 'cut_Very Good',
 'clarity_I1',
 'clarity_IF',
 'clarity_SI1',
 'clarity_SI2',
 'clarity_VS1',
 'clarity_VS2',
 'clarity_VVS1',
 'clarity_VVS2']

In [7]:
train_frame = train_high
model_name = 'diamonds_high'

aml_high = H2OAutoML(max_runtime_secs = runtime, project_name = f'{model_name}', keep_cross_validation_models=True)
aml_high.train(x=xTrainCols, y = 'price', training_frame = train_frame)
pred_high = aml_high.leader.predict(test_high)
display(pred_high)
display(aml_high.leaderboard)

AutoML progress: |████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
15426.9
16224.1
15857.7
15554
15564.1
14982.2
15330.4
14391.6
15616.2
16207.8


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_AutoML_20200301_143246,4.42018e+06,2102.42,4.42018e+06,1746.54,0.14893
XGBoost_3_AutoML_20200301_143246,4.43703e+06,2106.43,4.43703e+06,1755.01,0.149262
StackedEnsemble_AllModels_AutoML_20200301_143246,4.45012e+06,2109.53,4.45012e+06,1748.55,0.1492
XGBoost_grid__1_AutoML_20200301_143246_model_4,4.49906e+06,2121.1,4.49906e+06,1766.58,0.150977
GBM_grid__1_AutoML_20200301_143246_model_35,4.50676e+06,2122.91,4.50676e+06,1770.25,0.151458
GBM_2_AutoML_20200301_143246,4.51368e+06,2124.54,4.51368e+06,1763.32,0.150788
GBM_grid__1_AutoML_20200301_143246_model_37,4.52908e+06,2128.16,4.52908e+06,1785.34,0.152594
GBM_1_AutoML_20200301_143246,4.5296e+06,2128.29,4.5296e+06,1766.92,0.151453
GBM_grid__1_AutoML_20200301_143246_model_31,4.53335e+06,2129.17,4.53335e+06,1759.22,0.151371
XGBoost_grid__1_AutoML_20200301_143246_model_2,4.55803e+06,2134.96,4.55803e+06,1773.25,0.151158


In [8]:
tableHighPreds = test_high['id'].cbind(pred_high)
tableHighPreds = tableHighPreds.rename(columns={'predict':'price'})

In [9]:
train_frame = train_low
model_name = 'diamonds_low'

aml_low = H2OAutoML(max_runtime_secs = runtime, project_name = f'{model_name}', keep_cross_validation_models=True)
aml_low.train(x=xTrainCols, y = 'price', training_frame = train_frame)
pred_low = aml_low.leader.predict(test_low)
display(pred_low)
display(aml_low.leaderboard)

AutoML progress: |████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
4528.73
1349.84
6116.22
1508.97
2444.34
1866.56
2144.11
3231.82
13174.2
3351.14


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20200301_143752,777032,881.494,777032,465.78,0.159941
StackedEnsemble_BestOfFamily_AutoML_20200301_143752,778214,882.164,778214,465.353,0.159268
XGBoost_1_AutoML_20200301_143752,790013,888.827,790013,466.448,0.158266
XGBoost_3_AutoML_20200301_143752,792305,890.115,792305,476.693,0.170006
GBM_3_AutoML_20200301_143752,794128,891.139,794128,470.499,0.161753
GBM_2_AutoML_20200301_143752,794635,891.423,794635,472.354,0.163894
GBM_1_AutoML_20200301_143752,795479,891.896,795479,474.931,nan
DRF_1_AutoML_20200301_143752,801014,894.994,801014,468.793,0.159395
GBM_4_AutoML_20200301_143752,802360,895.745,802360,469.637,0.160809
XGBoost_2_AutoML_20200301_143752,804025,896.674,804025,468.353,0.157513


In [10]:
tableLowPreds = test_low['id'].cbind(pred_low)
tableLowPreds = tableLowPreds.rename(columns={'predict':'price'})

In [11]:
combinedTable = tableLowPreds.concat(tableHighPreds, axis=0)
combinedTable.shape

(13449, 2)

In [12]:
h2o.export_file(frame=combinedTable,path='../predictions/13-Auto-ML-Split-data-colorAsInt-restAsDummies-v4.csv')

Export File progress: |███████████████████████████████████████████████████| 100%
